### Bibliotecas utilizadas

- pandas: ferramenta para manipulação e análise de dados;
- plotly: utilitário especializado para plotagem de gráficos;
- pandasql: ferramenta para executar consultas sql em objetos do tipo pandas.Dataframe utilizando auxilio do SQLite;

In [1]:
import pandas as _pd
import plotly.graph_objects as _go
import plotly.express as _px
from pandasql import sqldf as sql

### Caminho do arquivo

In [2]:
dataset_path = 'data/Salary.csv'

### Função para leitura de arquivos csv

Essa função tem o recebe o caminho de um arquivo csv separados por vírgula e com codificação `utf-8`.
E retorna um objeto do tipo `pandas.Dataframe` com os dados do arquivo csv.

In [3]:
def read_file(path:str):
    tbl = _pd.read_csv(dataset_path, sep=',', encoding='utf-8')
    
    return tbl 

### Criação do dataset
Foi utilizada a função  `read_file` para fazer a leitura do arquivo e gerar o dataset

In [4]:
tbl = read_file(dataset_path)
tbl

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior
0,32.0,Male,1,Software Engineer,5.0,90000.0,UK,White,0
1,28.0,Female,2,Data Analyst,3.0,65000.0,USA,Hispanic,0
2,45.0,Male,3,Manager,15.0,150000.0,Canada,White,1
3,36.0,Female,1,Sales Associate,7.0,60000.0,USA,Hispanic,0
4,52.0,Male,2,Director,20.0,200000.0,USA,Asian,0
...,...,...,...,...,...,...,...,...,...
6679,49.0,Female,3,Director of Marketing,20.0,200000.0,UK,Mixed,0
6680,32.0,Male,0,Sales Associate,3.0,50000.0,Australia,Australian,0
6681,30.0,Female,1,Financial Manager,4.0,55000.0,China,Chinese,0
6682,46.0,Male,2,Marketing Manager,14.0,140000.0,China,Korean,0


### Tratamento dos dados
Para facilitar a visualização e utilização dos dados foi feita a tradução das colunas para o português.
A função `rename` recebe um dicionário com o de-para e retorna um `pandas.Dataframe`.

In [5]:
tbl = tbl.rename(
    columns={
        'Age': 'idade',
        'Gender': 'genero',
        'Education Level': 'nivel_educacao',
        'Job Title': 'cargo',
        'Years of Experience': 'anos_experiencia',
        'Salary': 'salario',
        'Country': 'pais',
        'Race': 'etnia',
        'Senior': 'senior'
    }
)
tbl

,idade,genero,nivel_educacao,cargo,anos_experiencia,salario,pais,etnia,senior
0,32.0,Male,1,Software Engineer,5.0,90000.0,UK,White,0
1,28.0,Female,2,Data Analyst,3.0,65000.0,USA,Hispanic,0
2,45.0,Male,3,Manager,15.0,150000.0,Canada,White,1
3,36.0,Female,1,Sales Associate,7.0,60000.0,USA,Hispanic,0
4,52.0,Male,2,Director,20.0,200000.0,USA,Asian,0
...,...,...,...,...,...,...,...,...,...
6679,49.0,Female,3,Director of Marketing,20.0,200000.0,UK,Mixed,0
6680,32.0,Male,0,Sales Associate,3.0,50000.0,Australia,Australian,0
6681,30.0,Female,1,Financial Manager,4.0,55000.0,China,Chinese,0
6682,46.0,Male,2,Marketing Manager,14.0,140000.0,China,Korean,0


### Indicador 1: Quais países concentram os maiores salários?
1. Para atender o seguinte indicador foi utilizada a consulta sql armazenada na variável `query`. Na qual, retorna a média dos salários agrupada por país de forma decrescente.
2. A instrução `sql` executa a consulta em cima do dataset armazenado na variável `tbl` e armazena na variável `result`.
3. A função `_px.bar` faz a construção de uma figura de um gráfico de barra horizontal com base nos parâmetros fornecidos. Onde o eixo x é a média dos salários e o eixo y é a descrição país.
4. A chamada `fig.show()` faz a figura aparecer na tela.

In [14]:
query = """
    SELECT pais, AVG(salario) salario
    FROM  dataset 
    GROUP BY pais
    ORDER BY salario ASC
"""

result = sql(query, env={'dataset': tbl})
fig = _px.bar(
    result, 
    x='salario', 
    y='pais',
    text_auto=True, # apresenta rótolos nas colunas do gráficos
    height=500,     # altura da imagem
    width=800       # largura da imagem
)
fig.show()


### Indicador 2: Qual a média de salário por etnia de cada país?
1. Para atender o seguinte indicador foi utilizada a consulta sql armazenada na variável `query`. Na qual, retorna a média dos salários agrupada por país e etnia de forma decrescente.
2. A instrução `sql` executa a consulta em cima do dataset armazenado na variável `tbl` e armazena na variável `result`.
3. A função `_go.Figure` faz a construção de uma figura de uma tabela, onde os dados são armazenados na variável `result`, o cabeçalho recebe uma lista dos nomes das colunas contida no objeto `tbl`.
4. As células recebem uma lista de objetos do tipo `pandas.Series` no qual representam as colunas país, etnia e salário.
5. A chamada `fig.show()` faz a figura aparecer na tela.

OBS: os parâmetros `fill_color` e `align` definem o esquema de cor e o alinhamento do texto respectivamente.

In [7]:
query = """
    SELECT pais, etnia, AVG(salario) salario
    FROM  dataset 
    GROUP BY pais, etnia
    ORDER BY salario DESC
"""

result = sql(query, env={'dataset': tbl})
fig = _go.Figure(data=[_go.Table(
    header=dict(values=list(result.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[result['pais'], result['etnia'], result['salario']],
               fill_color='lavender',
               align='left'))
])
fig.show()

### Indicador 3: Qual a distribuição de empregos por país?
1. Para atender o seguinte indicador foi utilizada a consulta sql armazenada na variável `query`. Na qual, retorna a contagem de cargos agrupados por país e cargo de forma decrescente.
2. A instrução `sql` executa a consulta em cima do dataset armazenado na variável `tbl` e armazena na variável `result`.
3. A função `_go.Figure` faz a construção de uma figura de uma tabela, onde os dados são armazenados na variável `result`, o cabeçalho recebe uma lista dos nomes das colunas contida no objeto `tbl`.
4. As células recebem uma lista de objetos do tipo `pandas.Series` no qual representam as colunas país, cargo e contagens de cargo por país.
5. A chamada `fig.show()` faz a figura aparecer na tela.

OBS: os parâmetros `fill_color` e `align` definem o esquema de cor e o alinhamento do texto respectivamente.

In [8]:
query = """
    SELECT pais, cargo, COUNT(cargo) quantidade
    FROM  dataset 
    GROUP BY pais, cargo
    ORDER BY 3 DESC
"""

result = sql(query, env={'dataset': tbl})
fig = _go.Figure(data=[_go.Table(
    header=dict(values=list(result.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[result['pais'], result['cargo'], result['quantidade']],
               fill_color='lavender',
               align='left'))
])
fig.show()

### Indicador 4: Qual é a média salarial por gênero?
1. Para atender o seguinte indicador foi utilizada a consulta sql armazenada na variável `query`. Na qual, retorna a média dos salários agrupada por gênero de forma decrescente.
2. A instrução `sql` executa a consulta em cima do dataset armazenado na variável `tbl` e armazena na variável `result`.
3. A função `_px.pie` faz a construção de uma figura de um gráfico de pizza com base nos parâmetros fornecidos. Onde os valores são o salário e as categorias são o gênero.
4. A chamada `fig.update_traces` faz customização da apresentação dos valores e dos traços dentro do gráfico.
5. A chamada `fig.show()` faz a figura aparecer na tela.

In [15]:
query = """
    SELECT genero, AVG(salario) salario
    FROM  dataset 
    GROUP BY genero
"""
result = sql(query, env={'dataset': tbl})
fig = _px.pie(
    result, 
    values='salario',
    names='genero',
    color_discrete_sequence=_px.colors.sequential.RdBu, # configura cores
    height=500,     # altura da imagem
    width=800       # largura da imagem
)
fig.update_traces(textposition='inside', textinfo='value+label+percent')

fig.show()

### Indicador 5: quais são os cargos que demandam mais anos de experiência?
1. Para atender o seguinte indicador foi utilizada a consulta sql armazenada na variável `query`. Na qual, retorna a média dos anos de experiência agrupada por cargo de forma decrescente.
2. A instrução `sql` executa a consulta em cima do dataset armazenado na variável `tbl` e armazena na variável `result`.
3. A função `_px.bar` faz a construção de uma figura de um gráfico de barra vertical com base nos parâmetros fornecidos. Onde o eixo x são as descrições do cargo e o eixo y é a média dos anos de experiência.
4. A chamada `fig.show()` faz a figura aparecer na tela.

In [17]:
query = """
    SELECT cargo, AVG(anos_experiencia) anos_experiencia
    FROM  dataset 
    GROUP BY 1
    ORDER BY 2 DESC
"""

result = sql(query, env={'dataset': tbl})
fig = _px.bar(
    result.head(15), 
    x='cargo', 
    y='anos_experiencia',
    text_auto=True, # apresenta rótolos nas colunas do gráficos
    height=500,     # altura da imagem
    width=800       # largura da imagem
)
fig.show()
